In [ ]:
"""    
    Indicador: Hectareaje X Hito.
"""

<h3>Importación Librerías</h3>

In [ ]:
import pandas as pd
import os
import seaborn as sns

os.environ["CRYPTOGRAPHY_OPENSSL_NO_LEGACY"] = "yes"

# ? Se usa para generar gráficos
import matplotlib.pyplot as plt

# ? Se usa para determina la fecha de ejecución del proceso dentro del export de los resultados estadísticos
from datetime import datetime

# ? Conexión a BD Postgres. Se usa dado que pandas no acepta sino este paquete para la conexión.
from sqlalchemy import create_engine, text

from datetime import datetime as df

from arcgis.features import GeoAccessor, GeoSeriesAccessor
import arcpy

from pathlib import Path

# ** Librerías propias
import funcion_relacion_terrenos_w_ui
import funcion_hectareaje_general_ui

<h3>Parametrización de Variables</h3>

In [ ]:
RUTA_TERRENOS_RECONO_TRAMITES_ATENCION_BCGS = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E1_Alistamiento_Diagnostico\3_Disposicion\1.BD_Consolidada\BD_Consolidada_Lote4.gdb\Analitica_UT_Consolidada\TERRENOS_RECONOCIMIENTO_TRAMITES_ATENCION_BCGS"
DIRECTORIO_XLSX_2_2_4 = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\2_2_4_Hectareaje_X_Hito\zReportes\Seguimiento_Diario"

columnas_terrenos_w_ui = ['codigo','radicado','municipio','proceso','area_ha_cmt12','ID_UI','Zona_UI','Nombre_UI', 'Area_Ha_CMT12', 'Meta_Hito']
columnas_reducion_indicador_1 = ['Hito','Area_Ha_Contractual','hectareaje_visitado_bcgs']

fecha_actual = df.now()
fecha_directorio = str(fecha_actual.strftime("%Y-%m-%d"))

nombre_xlsx_2_2_4 = str(fecha_directorio) + '_indicador_hectareaje_avance_por_hito.xlsx'
RUTA_2_2_4 = os.path.join(DIRECTORIO_XLSX_2_2_4, nombre_xlsx_2_2_4)

ruta_fcTerrenos = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E1_Alistamiento_Diagnostico\3_Disposicion\1.BD_Consolidada\BD_Consolidada_Lote4.gdb\Analitica_UT_Consolidada\TERRENO_UNIFICADO_w_GIS"
ruta_bdScrach = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\2_2_0_Identificacion_Predial_Total_Ha_Actualizadas\Base_Datos\Terrenos_w_UI.gdb"
ruta_fcUI = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\O_Apoyo_Construccion_UIntervencion\UI_Finales\UI_Finales.gdb\UI_Unificadas\UI_Unificadas"

<h3>Ejecución de Funciones</h3>

In [ ]:

# ** Generación de ruta a la relación terreno ui
ruta_tlbTerrenoUI = funcion_relacion_terrenos_w_ui.funcion_terreno_w_ui(ruta_fcTerrenos, ruta_bdScrach, ruta_fcUI)

# ** Genera los DF de los estadísticos generales contractuales de hito vs Ha
df_hitos_ut, df_unidades_intervencion_hitos = funcion_hectareaje_general_ui.funcion_hectareaje_general_hito(ruta_fcUI)

<h3>Actualización</h3>

In [ ]:

# ** Lectura del FC Generado en 2.2.0 {Terrenos Georreferenciados asociados a la Gestión BCGS}
df_terrenosAtencionBCGS = pd.DataFrame.spatial.from_featureclass(RUTA_TERRENOS_RECONO_TRAMITES_ATENCION_BCGS)

# ? Se filtra Actualización
df_terrenosReconocimientoBCGS = df_terrenosAtencionBCGS[df_terrenosAtencionBCGS['proceso']=='Actualización']
# ? Copia de DF
df_terrenosReconocimientoBCGS = df_terrenosReconocimientoBCGS.copy()
# ? Se extrae organizados propio de columnas
df_terrenosReconocimientoBCGS['id_actividad'] = df_terrenosReconocimientoBCGS['estado_tramite'].str[0:1].astype(int)
# ? Se filtra por indicador de proceso
df_terrenosReconocimientoBCGS = df_terrenosReconocimientoBCGS[df_terrenosReconocimientoBCGS['id_actividad'] >= 6]

# ** Lectura del Relación Terreno - UI
df_relTerrenosUI = pd.DataFrame.spatial.from_table(ruta_tlbTerrenoUI)
# ** FC UI Completa
df_UI = pd.DataFrame.spatial.from_featureclass(ruta_fcUI)

# ** Unión, Terrenos con Tabla Relación Terrenos Vs UI
df_terrenosBCGS = pd.merge(left=df_terrenosReconocimientoBCGS,
    right=df_relTerrenosUI,
    left_on="codigo",
    right_on="codigo",
    how="inner"
    )

# ** Unión, Terrenos con ID_UI con FC de UI para completar datos de Hito
df_terrenosBCGS_w_UI = pd.merge(left=df_terrenosBCGS,
    right=df_UI,
    left_on = 'id_ui',
    right_on = 'ID_UI',
    how="inner"
    )

# ** Reducción de Columnas para mejorar análisis
df_terrenosBCGS_w_UI_reducido = df_terrenosBCGS_w_UI[columnas_terrenos_w_ui]
df_terrenosBCGS_w_UI_reducido = df_terrenosBCGS_w_UI_reducido.rename(columns={'Area_Ha_CMT12':'Area_Ha_CTM12_UI', 'area_ha_cmt12':'area_ha_ctm12_predio'})
print(f'{df_terrenosBCGS_w_UI_reducido.shape[0]}')
df_terrenosReconocimientoBCGS

<h3>Indicadores</h3>

In [ ]:

# ** Indicador 1: Total Gestión BSCG (Activo) por Hito
df_terrenosBCGS_w_UI_indicador_1 = pd.DataFrame(df_terrenosBCGS_w_UI_reducido.groupby(['Meta_Hito'])['area_ha_ctm12_predio'].sum().round(3)).reset_index().rename(columns={'area_ha_ctm12_predio':'hectareaje_visitado_bcgs'})

# ** Indicador 2: Total Gestión BSCG (Activo) por Hito y UI
df_terrenosBCGS_w_UI_indicador_2 = pd.DataFrame(df_terrenosBCGS_w_UI_reducido.groupby(['Meta_Hito','ID_UI','Area_Ha_CTM12_UI'])['area_ha_ctm12_predio'].sum().round(3)).reset_index().rename(columns={'area_ha_ctm12_predio':'hectareaje_visitado_bcgs'})

In [ ]:
df_terrenosBCGS_w_UI_indicador_1

In [ ]:
df_indicador_1_contrato = pd.merge(left=df_hitos_ut,
    right=df_terrenosBCGS_w_UI_indicador_1,
    left_on = 'Hito',
    right_on = 'Meta_Hito',
    how="inner")

df_indicador_1_contrato_reducido = df_indicador_1_contrato[columnas_reducion_indicador_1]

df_indicador_1_contrato_reducido = df_indicador_1_contrato_reducido.copy()

df_indicador_1_contrato_reducido['porcentaje_visitado_bcgs'] = ((df_indicador_1_contrato_reducido['hectareaje_visitado_bcgs'] / df_indicador_1_contrato_reducido['Area_Ha_Contractual']) * 100).round(4)
df_indicador_1_contrato_reducido['hectareaje_predios_finalizados'] = 0

df_indicador_1_contrato_reducido

In [ ]:
df_terrenosBCGS_w_UI_indicador_2['Area_Ha_CTM12_UI'] = df_terrenosBCGS_w_UI_indicador_2['Area_Ha_CTM12_UI'].round(3)
df_terrenosBCGS_w_UI_indicador_2['porcentaje_visitado_bcgs'] = ((df_terrenosBCGS_w_UI_indicador_2['hectareaje_visitado_bcgs'] / df_terrenosBCGS_w_UI_indicador_2['Area_Ha_CTM12_UI']) * 100).round(4)
df_terrenosBCGS_w_UI_indicador_2['hectareaje_predios_finalizados'] = 0

df_terrenosBCGS_w_UI_indicador_2

<h3>Generación XLSX</h3>

In [ ]:
with pd.ExcelWriter(RUTA_2_2_4, engine='xlsxwriter') as writer:
    df_indicador_1_contrato_reducido.to_excel(writer, sheet_name='Indicador_1', index=False)
    df_terrenosBCGS_w_UI_indicador_2.to_excel(writer, sheet_name='Indicador_2', index=False)

print("Finaliza la exportación del XLSX")